# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()# проверяем данные 
print(df.duplicated().value_counts())#изучаем данные на наличие дубликатов
print(df.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
False    3214
dtype: int64
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


Открываем файл с помощью библиотеки Pandas, проверяем типы данных, пропущенных значений нет, дубликатов нет.

## Разбейте данные на выборки

In [2]:
df, df_test = train_test_split(df, test_size = 0.2, random_state = 12345)# выделяем тестовую выборку
target_test = df_test['is_ultra'] #выделяем целевые значения
features_test = df_test.drop(['is_ultra'], axis = 1)
target = df['is_ultra'] #выделяем целевые значения
features = df.drop(['is_ultra'], axis = 1)# выделяем параметры 
train_target, valid_target, train_features, valid_features = train_test_split(target, features, test_size = 0.25,
                                                                              random_state = 12345)# делим выборку на тренировочную и валидационную


Делим выборку на тестовую, валидационную и тренировочную в соотношении 20%х20%х60%. Определяем параметры и целевые значения.

## Исследуйте модели

In [3]:
best_model = None
best_result = 0
for est in range(1, 101, 10):
    for depth in range (1, 21):# проверяем циклом количество деревьев в случанйном лесе с наилучшими параметрами accuracy
        model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth = depth) 
        model.fit(train_features, train_target) 
        result = model.score(valid_features, valid_target) 
        if result > best_result:
            best_model = model
            best_result = result
print('Параметры лучшей модели на основе случайного  леса:', best_model,'Значение Accuracy', best_result)

Параметры лучшей модели случайного  леса: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=15, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=21,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False) Значение Accuracy 0.8009331259720062


Проводим обучение модели на основе случайного леса, подбираем оптимальное значение деревьев в промежутке от 1 до 101 с шагом в 10. И глубины деревьев в промежутке от 1 до 20. Считаем значение Accuracy для всех гиперпараметров, находим наиболее подходящее. Лучшим полученным на валидационной выборке значением является 80% точности. Сохраняем модель в переменную best_model.

In [4]:
best_model1 = None
best_result1 = 0
for i in range (1,21):
    model1 = DecisionTreeClassifier(random_state=12345, max_depth = i)
    model1.fit(train_features, train_target) 
    result1 = model1.score(valid_features, valid_target) 
    if result1 > best_result1:
        best_model1 = model1
        best_result1 = result1
print('Параметры лучшей модели на основе дерева решений:', best_model1,'Значение Accuracy', best_result1)

Параметры лучшей модели дерева решений: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best') Значение Accuracy 0.7744945567651633


Проводим обучение модели на основе дерева решений, подбираем оптимальное значение глубины деревьев в промежутке от 1 до 20. Считаем значение Accuracy для всех гиперпараметров, находим наиболее подходящее. Лучшим полученным на валидационной выборке значением является 77% точности. Сохраняем модель в переменную best_model1.

In [5]:
model2 = LogisticRegression(random_state=12345)
model2.fit(train_features, train_target) 
pred = model2.predict(valid_features)
result2 = model2.score(valid_features, valid_target) 
print('Значение Accuracy у логистической регрессии :', result2)

Значение Accuracy у логистической регрессии : 0.6967340590979783


Проводим обучение модели на основе логистической регрессии. Полученным на валидационной выборке значением является 69% точности. Сохраняем модель в переменную model2.

Исходя из проведенных тестов моделей на валидацинной выборке, лидером является случайный лес с результатом 80% правильных предсказаний. Дерево решений тоже хорошо себя показало с результатом в 77% правильных предсказаний. Логистическая регрессия сильно отстает с результатом 70%.

## Проверьте модель на тестовой выборке

In [6]:

print('Результат проверки лучшей модели на тестовой выборке:',best_model.score(features_test, target_test))# проверяем качество модели на тестовой выборке
print('Результат проверки второй по качеству модели на тестовой выборке:',best_model1.score(features_test, target_test))

Результат проверки лучшей модели на тестовой выборке: 0.7822706065318819
Результат проверки второй по качеству модели на тестовой выборке: 0.7884914463452566


В результате проведения тестирования лучшей модели на тестовой выборке, получили результат 78%. Для интереса проверили результат примения на тестовой выборке лучшей модели из класса Дерево решений, результаты получились сопостовимыми. На первый взгляд, выглядит неплохо, но нужно еще проверить модель на адекватность.

## (бонус) Проверьте модели на адекватность

In [7]:
from sklearn.dummy import DummyClassifier #добавляем DummyClassifier из библиотеки sklearn
frequent_cls = DummyClassifier(strategy = 'most_frequent').fit(train_features, train_target)# обучаем DummyClassifier на нашей обучающей выборке с параметром "most_frequent"
print('Значение Accuracy для DummyClassifier который предсказывает самый частый вариант класса', frequent_cls.score(features_test, target_test)) #“stratified”, “most_frequent”, “prior”, “uniform”, “constant”
stratified_cls = DummyClassifier(strategy = 'stratified').fit(train_features, train_target)
print('Значение Accuracy для DummyClassifier который предсказывает классы исходя из класса обучающей выборки', stratified_cls.score(features_test, target_test))
uniform_cls = DummyClassifier(strategy = 'uniform').fit(train_features, train_target)
print('Значение Accuracy для DummyClassifier который предсказывает классы поровну', uniform_cls.score(features_test, target_test))

Значение Accuracy для DummyClassifier который предсказывает самый частый вариант класса 0.6951788491446346
Значение Accuracy для DummyClassifier который предсказывает классы исходя из класса обучающей выборки 0.5629860031104199
Значение Accuracy для DummyClassifier который предсказывает классы поровну 0.5116640746500778


Исходя из проведенного тестирования, делаем вывод, что выбранная модель отвечает критериям адекватности, случайно результат 78% правильных предсказаний получить не удалось. Просто проставление самого популярного класса дала результат в 70% правильных предсказаний, результат сопоставим с использованием логистической регрессии, делаем вывод, что данная модель в данном случае была правильно отброшена в ходе выбора модели.

По итогу проведенной работы, можно заключить, что на практике возможно использование модели на основе рандомоного леса и дерева решений, поскольку на тестовой выборке они показали сходный результат. Валидационная выборка показала преимущество рандомного леса (3%), однако в условиях ограниченных вычислительных ресурсов дерево решений тоже может применяться. 